In [2]:
import numpy as np
import cv2
from keras.models import Sequential
from keras.layers import Dense,Dropout,Flatten
from keras.layers import Conv2D
from keras.optimizers import Adam
from keras.layers import MaxPooling2D
from keras.preprocessing.image import ImageDataGenerator

In [3]:
train_dir='data/train'
val_dir='data/test'
train_datagen=ImageDataGenerator(rescale=1./255,
                                 rotation_range=20, # rotate images randomly up to 20 degrees
                                  zoom_range=0.2, # zoom in up to 20%
                                horizontal_flip=True, # flip images horizontally
                                 width_shift_range=0.1, # shift images horizontally up to 10%
                                 height_shift_range=0.1 # shift images vertically up to 10%
)
val_datagen=ImageDataGenerator(rescale=1./255)

train_generator=train_datagen.flow_from_directory(
    train_dir,
    target_size=(48,48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical')
validation_generator=val_datagen.flow_from_directory(
    val_dir,
    target_size=(48,48),
    batch_size=64,
    color_mode='grayscale',
    class_mode='categorical'
)

Found 28709 images belonging to 7 classes.
Found 7178 images belonging to 7 classes.


In [4]:
emotion_model=Sequential()
emotion_model.add(Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=(48,48,1)))
emotion_model.add(Conv2D(64,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Conv2D(128,kernel_size=(3,3),activation='relu'))
emotion_model.add(MaxPooling2D(pool_size=(2,2)))
emotion_model.add(Dropout(0.25))
emotion_model.add(Flatten())
emotion_model.add(Dense(1024,activation='relu'))
emotion_model.add(Dropout(0.5))
emotion_model.add(Dense(7,activation='softmax'))

In [5]:
emotion_model.compile(loss='categorical_crossentropy',optimizer=Adam(learning_rate=0.0001,decay=1e-6),metrics=['accuracy'])
emotion_model_info=emotion_model.fit(
    train_generator,
    steps_per_epoch=28709//64,
    epochs=30,
    validation_data=validation_generator,
    validation_steps=7178//64
)

Epoch 1/30
448/448 [==============================] - 443s 978ms/step - loss: 1.8020 - accuracy: 0.2600 - val_loss: 1.7178 - val_accuracy: 0.3386
Epoch 2/30
448/448 [==============================] - 449s 1s/step - loss: 1.6428 - accuracy: 0.3591 - val_loss: 1.5733 - val_accuracy: 0.4065
Epoch 3/30
448/448 [==============================] - 444s 990ms/step - loss: 1.5449 - accuracy: 0.4063 - val_loss: 1.4757 - val_accuracy: 0.4403
Epoch 4/30
448/448 [==============================] - 459s 1s/step - loss: 1.4705 - accuracy: 0.4334 - val_loss: 1.4186 - val_accuracy: 0.4639
Epoch 5/30
448/448 [==============================] - 451s 1s/step - loss: 1.4107 - accuracy: 0.4642 - val_loss: 1.3595 - val_accuracy: 0.4827
Epoch 6/30
448/448 [==============================] - 473s 1s/step - loss: 1.3601 - accuracy: 0.4831 - val_loss: 1.3213 - val_accuracy: 0.5089
Epoch 7/30
448/448 [==============================] - 504s 1s/step - loss: 1.3115 - accuracy: 0.5055 - val_loss: 1.2962 - val_accuracy: 

In [6]:
emotion_model.save_weights('model.h5')


In [7]:
cv2.ocl.setUseOpenCL(False)
emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral",5: "Sad" , 6: "Surprised"}

In [8]:
emoji_dist={0:"data/emojis/angry.png",1:"data/emojis/disgusted.png",2:"data/emojis/fearful.png",3:"data/emojis/happy.png",4:"data/emojis/neutral.png",5:"data/emojis/sad.png",6:"data/emojis/surpriced.png"}

In [10]:
cap = cv2.VideoCapture(0)
while True:
    ret, frame = cap.read()
    if not ret:
        break
   
    
    bounding_box = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')
    #{
    frame = cv2.fastNlMeansDenoisingColored(frame, None, 10, 10, 7, 21)
    frame = cv2.GaussianBlur(frame, (5, 5), 0)
   #}
    gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    num_faces = bounding_box.detectMultiScale(gray_frame,scaleFactor=1.3, minNeighbors=5)
    for (x, y, w, h) in num_faces:
        cv2.rectangle(frame, (x, y-50), (x+w, y+h+10), (255, 0, 0), 2)
        roi_gray_frame = gray_frame[y:y + h, x:x + w]
        cropped_img = np.expand_dims(np.expand_dims(cv2.resize(roi_gray_frame, (48, 48)), -1), 0)
        emotion_prediction = emotion_model.predict(cropped_img)
        maxindex = int(np.argmax(emotion_prediction))
        emoji_dist={0:"data/emojis/angry.png",1:"data/emojis/disgusted.png",2:"data/emojis/fearful.png",3:"data/emojis/happy.png",4:"data/emojis/neutral.png",5:"data/emojis/sad.png",6:"data/emojis/surpriced.png"}
       
       
        image=cv2.imread(emoji_dist[maxindex])

        cv2.imshow('Emojify',image)
        emotion_dict = {0: "Angry", 1: "Disgusted", 2: "Fearful", 3: "Happy", 4: "Neutral", 5: "Sad", 6: "Surprised"}
        cv2.putText(frame, emotion_dict[maxindex], (x+20, y-60), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 255, 255), 2, cv2.LINE_AA)
    cv2.imshow('Video', cv2.resize(frame,(1200,860),interpolation = cv2.INTER_CUBIC))
    if cv2.waitKey(1) & 0xFF == ord('q'):
       cap.release()
       cv2.destroyAllWindows()
       break

1/1 [==============================] - 0s 46ms/step


In [ ]:
emotion_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        320       
                                                                 
 conv2d_1 (Conv2D)           (None, 44, 44, 64)        18496     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 22, 22, 64)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 22, 22, 64)        0         
                                                                 
 conv2d_2 (Conv2D)           (None, 20, 20, 128)       73856     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 10, 10, 128)      0         
 2D)                                                    